# Import modules.

In [7]:
%%capture
%load_ext autoreload
%autoreload 2


import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import holoviews as hv
import csv
import math

from scipy.stats import zscore
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.core.display import display

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
minian_path = "."

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

# Specify data location.

In [5]:
%%capture
sys.path.insert(0, minian_path)
from minian.utilities import (
    open_minian,
    match_information,
    match_path,
    TaskAnnotation
)

NameError: name 'minian_path' is not defined

In [6]:
# Set up Initial Basic Parameters#

dpath = "/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA058_D1/2023_05_05/11_02_42/Miniscope_2/ALP_session"
mouseID, day, session = match_information(dpath)
mouse_path, video_path = match_path(dpath)
timeStamp = os.path.join(video_path,"timeStamps.csv")
minian_ds_path = os.path.join(dpath, "minian")
session = 'SALP'
output_dpath = "/N/project/Cortical_Calcium_Image/data_analysis"
# output_path = os.path.join(output_dpath, mouseID,day,session)

n_workers = int(os.getenv("MINIAN_NWORKERS", 4))
# behavior_data=pd.read_csv(os.path.join(mouse_path, mouseID+"_"+day+"_behavior_ms.csv"),sep=',')
# behavior_data=pd.read_csv(os.path.join(mouse_path, mouseID+"_"+day+"_"+session+"_behavior_ms.csv"),sep=',')

if(os.path.exists(output_path) == False):
    os.makedirs(output_path)

NameError: name 'match_information' is not defined

In [ ]:
dpath = os.path.abspath(dpath)
hv.notebook_extension("bokeh", width=75)

# Load data

In [ ]:
cluster = LocalCluster(
    n_workers=n_workers,
    memory_limit="5GB",
    resources={"MEM": 1},
    threads_per_worker=2,
    dashboard_address=":8787",
)
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)

## Data structure of Minian outputs
Here is the output of Minian. It is an xarray Dataset that contains coordinate labels for different aspects of the data. The way you typically reference the data is the same as you would a Python dictionary. That is, using data['key']. For example, data['A']. See more here: https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset. 

The most common variables you will be accessing are A, S, and sometimes C. 
data['A'] contains the spatial footprints of the detected neurons. 
data['S'] contains the deconvolved spikes.
data['C'] contains the modeled calcium traces. 

Each of these are arranged such that their first dimension is unit_id. So for example, in order to access the first neuron's spatial footprint you would call data['A'].sel(unit_id=0). To access the second, data['A'].sel(unit_id=1), etc.

In [ ]:
# print(minian_ds_path)
data = open_minian(minian_ds_path)
data

In [ ]:
data['C'].sel(unit_id=data['unit_id'])

# Store/open data in different formats

## Converting the data into other formats is relatively simple. See below for examples.

In [ ]:
# Matlab-readable
data.to_netcdf(os.path.join(dpath, "minian_dataset.nc"))

In [ ]:
# Pandas DataFrame
df = data['C'].to_pandas()
df

In [ ]:
# Save C value 
df.to_csv(os.path.join(output_path, 'minian_'+mouseID+"_"+day+"_"+session+'_C.csv'))

In [ ]:
df_S = data['S'].to_pandas()
df_S

In [ ]:
# Save S value
df_S.to_csv(os.path.join(output_path, 'minian_'+mouseID+"_"+day+"_"+session+'_S.csv'))

# Plotting data

## Calcium traces and spikes
Below are difference functions for plotting the modeled traces and spikes for individual neurons and groups of neurons. The features of these plots can be modified by the user.

In [ ]:
# # Plot individual traces.
# def plot_trace(data, neuron):
#     fig, C_ax = plt.subplots(figsize=(12,6))
#     S_ax = C_ax.twinx()
#     C_ax.plot(data['C'].sel(unit_id=neuron), color='royalblue')
#     C_ax.set_ylabel("C [modeled activity, A.U.]", color='royalblue')
#     C_ax.set_xlabel('Frame #')
#     C_ax.set_title(f'Neuron #{neuron}')

#     S_ax.plot(data['S'].sel(unit_id=neuron), color='r', alpha=0.5)
#     S_ax.set_ylabel("S [modeled activity, A.U.]", color='r')

In [ ]:
# neuron_number = 0
# plot_trace(data, neuron_number)

In [ ]:
#behavior_data from lever press
behavior_data

In [ ]:
[1,10,25]

In [ ]:
# Plot traces from multiple neurons
def plot_multiple_traces(data, neurons_to_plot, behavior_data, start_frame, session,shift_amount=0.4):
    shifts = [shift_amount * i for i in range(len(neurons_to_plot))]
    int:id_text_position
    if session == "S1":
        fifteen = behavior_data.loc[behavior_data['Time Stamp (ms)'] <= 900000]
        neuron_data=data.sel(frame=slice(0,len(fifteen)-1))
        id_text_position = -30
    elif session == "S4":
        fifteen = behavior_data.loc[behavior_data['Time Stamp (ms)'] >= 2700000]
        id_text_position = 2700000-30
#         print(len(fifteen))
#         print(len(data['C'].sel(unit_id=1)))
#         neuron_data=data.sel(frame=slice(len(data['C'].sel(unit_id=neurons_to_plot[0]))-len(fifteen),len(data['C'].sel(unit_id=neurons_to_plot[0])-1)))
        begin = len(data['C'].sel(unit_id=neurons_to_plot[0]))-len(fifteen) + data['frame'][0].values
        end = len(data['C'].sel(unit_id=neurons_to_plot[0])) + data['frame'][0].values - 1
#         print(begin)
#         print(end)
#         print(data['frame'])
#         print(data['frame'][0].values)
        neuron_data=data.sel(frame=slice(begin,end))
#         print(len(data['C'].sel(unit_id=neurons_to_plot[0])))
#         print(len(data['C'].sel(unit_id=neurons_to_plot[0]))-1)
#         print(len(fifteen))
#         print(len(neuron_data['C'].sel(unit_id=neurons_to_plot[0])))
    else:
        fifteen = behavior_data    
    y=shifts[-1]+10
    fig, ax = plt.subplots(figsize=(30.4,y))
    for shift, neuron in zip(shifts, neurons_to_plot):
#         if session == "S1":
#             trace = data['C'].sel(frame=slice(0,len(fifteen))).sel(unit_id=neuron)
#         else:
#             trace = data['C'].sel(frame=slice(len(data['C'])-1-len(fifteen),len(data['C']-1))).sel(unit_id=neuron)
#         print(trace)
        trace = neuron_data['C'].sel(unit_id=neuron)
        trace /= np.max(trace)
#         ax.autoscale()
        ax.text(id_text_position,shift,neuron)
        ax.plot(fifteen['Time Stamp (ms)'],trace + shift)
#         ax.plot(behavior_data.loc[behavior_data['Frame Number']<len(trace)]['Time Stamp (ms)'],trace + shift)
#         start_frame,start_frame+len(trace)
        ax.vlines(fifteen.loc[fifteen['RNFS'] > 0]['Time Stamp (ms)'],0,y-8,color="green")
        ax.vlines(fifteen.loc[fifteen['IALP'] > 0]['Time Stamp (ms)'],0,y-9,color="blue")
        ax.vlines(fifteen.loc[fifteen['ALP'] > 0]['Time Stamp (ms)'],0,y-9,color="red",alpha=0.5)
#         if session == "S1":
#             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=0) & (behavior_data['Frame Number']<(0+len(trace))) & (behavior_data['reinforcement']==1)]['Time Stamp (ms)'],0,y-8,color="green")
#             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=0) & (behavior_data['Frame Number']<(0+len(trace))) & (behavior_data['IALP']==1)]['Time Stamp (ms)'],0,y-9,color="blue")
#             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=0) & (behavior_data['Frame Number']<(0+len(trace))) & (behavior_data['ALP']==1)]['Time Stamp (ms)'],0,y-9,color="red",alpha=0.5)
#         else:
# #             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=len(data['C'])-1-len(fifteen)) & (behavior_data['Frame Number']<(start_frame+len(trace))) & (behavior_data['reinforcement']==1)]['Time Stamp (ms)'],0,y-8,color="green")
# #             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=len(data['C'])-1-len(fifteen)) & (behavior_data['Frame Number']<(start_frame+len(trace))) & (behavior_data['IALP']==1)]['Time Stamp (ms)'],0,y-9,color="blue")
# #             ax.vlines(behavior_data.loc[(behavior_data['Frame Number']>=len(data['C'])-1-len(fifteen)) & (behavior_data['Frame Number']<(start_frame+len(trace))) & (behavior_data['ALP']==1)]['Time Stamp (ms)'],0,y-9,color="red",alpha=0.5)
    ax.set_xlabel('Time Stamp (ms)')
    ax.set_ylabel('Neurons')
    ax.set_yticks([])
#     test=data['C'].sel(frame=slice(0,10)).sel(unit_id=0)
#     print(test)
    fig.savefig(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_trace_ms.pdf"))

In [ ]:
all_neurons=list(np.array(data['unit_id']))
# plot_multiple_traces(data, all_neurons,behavior_data,0,session)

## Spatial footprints
Same for spatial footprints. Functions can be customizable by us or the user.

In [ ]:
# Plot spatial footprints
def plot_footprints(data, neurons=None):
    if type(neurons) is int:
        neurons = [neurons]
    elif neurons is None:
        neurons = data['A']['unit_id']
    
    fov = np.zeros_like(data['A'].sel(unit_id=0))
    for neuron in neurons:
        fov += np.asarray(data['A'].sel(unit_id=neuron))
        
    fig, ax = plt.subplots(figsize=(12,12))
    ax.imshow(fov, origin='lower')
    ax.axis('image')

In [ ]:
# plot_footprints(data)

In [ ]:
# plot_footprints(data, good_looking_neurons)

# Basic analysis
Here are some basic descriptive statistics of the detected neurons. 

## Mean amplitudes
It is a good idea to use the S matrix, rather than the C matrix. This is because the S matrix contains deconvolved spikes whereas the C matrix is the modeled calcium trace, which includes the decay portion of the calcium signal. 

In [ ]:
# Histogram of mean amplitudes of all detected neurons. 
def histogram_mean_amplitude(data, nbins=50):
    mean_amplitudes = data['S'].mean(dim='frame')
    
    fig, ax = plt.subplots(figsize=(12,6))
    ax.hist(mean_amplitudes, bins=nbins)
    ax.set_xlabel('Mean amplitudes [A.U.]')
    ax.set_ylabel('# of neurons')
    
    return np.asarray(mean_amplitudes)

In [ ]:
mean_amplitudes = histogram_mean_amplitude(data, nbins=50)

In [ ]:
mean_amplitudes

## Event frequencies
It is even more important to use the S matrix here because the C matrix will include time bins where the calcium signal is decaying.

In [ ]:
# Histogram of event frequencies of all detected neurons. 
def histogram_event_freq(data, nbins=50):
    event_freq = [np.sum(n > 0)/len(n) for n in np.asarray(data['S'])]
    
    fig, ax = plt.subplots(figsize=(12,6))
    ax.hist(event_freq, bins=nbins)
    ax.set_xlabel('Event frequency [proportion of active frames]')
    ax.set_ylabel('# of neurons')
    
    return np.asarray(event_freq)

In [ ]:
event_freq = histogram_event_freq(data)

In [ ]:
event_freq

# Digital Dataset(DDS)


## AUC of S per minute

### merge file

In [ ]:
S_df=data['S'].to_pandas()
S_df=S_df.transpose()
S_df

In [ ]:
if session=="S1":
    new_pd=pd.merge(S_df,behavior_data,left_index=True,right_on='Frame Number',how='left')
else:
    temp_pd=behavior_data.tail(len(S_df))
    S_df.reset_index(drop=True, inplace = True)
    temp_pd.reset_index(drop=True, inplace=True)
    new_pd=pd.concat([S_df,temp_pd],axis=1)

In [ ]:
new_pd

###  Check frame interval

In [ ]:
new_pd.loc[new_pd['Time Stamp (ms)'].diff()>100]

## AUC of S per min

In [ ]:
new_pd['min']=new_pd['Time Stamp (ms)'].map(lambda x: math.floor(x/60000))
pd.set_option('display.min_rows',50)
# new_pd['min'] = new_pd['min']+2
res=new_pd.groupby('min').sum()
new_pd

In [ ]:
res.drop(columns=['Frame Number','Time Stamp (ms)','Buffer Index']).to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_AUC_of_S_per_minute.csv"))

In [ ]:
res

## Calculate z-score

In [ ]:
# Contains left and right ends
def calculate_window(analyzed_zscore_df, action , start, end):
    output_df = analyzed_zscore_df
    for i in range(start,end + 1):
        # only line 2 so the mean is the same to line 2.
        series_positive = analyzed_zscore_df[analyzed_zscore_df[action].shift(i) > 0].mean()
        series_positive.name = action+"_positive_W"+str(i)
        series_negative = analyzed_zscore_df[analyzed_zscore_df[action].shift(i) == 0].mean()
        series_negative.name = action+"_negative_W"+str(i)
        output_df = output_df.append(series_positive)
        output_df = output_df.append(series_negative)
    return output_df

In [ ]:
def calculate_zscore (session, input_df):
    neuron_ids = input_df.columns.values.tolist()
    neuron_ids = list(filter(lambda x:type(x) == int,neuron_ids))
    if session == "S1":
        analyzed_df = input_df.loc[0:14,:]
    elif session == "S4":
        analyzed_df = input_df.loc[45:59,:]
    neuron_value = analyzed_df.loc[:,neuron_ids]
    behavior_value = analyzed_df.loc[:,["RNFS","ALP","IALP","ALP_Timeout"]]
    zscore_df = zscore(neuron_value,axis = 0)
    sum_zscore_df = pd.merge(zscore_df,behavior_value,left_index=True,right_index=True)
    print(sum_zscore_df)
    output_zscore_df = calculate_window(sum_zscore_df, "ALP", -3, 3)
    output_zscore_df = calculate_window(output_zscore_df, "IALP", -3, 3)
    output_zscore_df = calculate_window(output_zscore_df, "RNFS", -3, 3)
    output_zscore_df = calculate_window(output_zscore_df, "ALP_Timeout", -3, 3)
    return output_zscore_df

### AUC zscore

In [ ]:
res

In [ ]:
AUC_zscore = calculate_zscore(session, res)

In [ ]:
AUC_zscore.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_AUC_zscore.csv"))

In [ ]:
zscore_summary_index_list = list(filter(lambda x: type(x) != int,AUC_zscore.index.tolist()))
AUC_zscore_summary = AUC_zscore.loc[zscore_summary_index_list,:]
AUC_zscore_summary

In [ ]:
def calculate_mean(Series,ngtv_near=0.1,ngtv_far=0,pstv_near=0.1,pstv_far=0):
    series=Series.sort_values()
    # total number
    total = len(Series)
    
    # ngtv number
    ngtv_series= series[series<0]
    ngtv_number = len(ngtv_series)
    
    # pstv number
    pstv_series = series[series>0]
    pstv_number = len(pstv_series)
    
    # number of ngtv be analyzed
    ngtv_series_analyzed = ngtv_series[round(len(ngtv_series)*ngtv_far):round(len(ngtv_series)*(1-ngtv_near))]
    ngtv_analyzed_number = len(ngtv_series_analyzed)
    
    # analyzed ngtv neuron ID
    ngtv_neurons_list = ngtv_series_analyzed.index.to_list()
    ngtv_neurons=','.join([str(i) for i in ngtv_neurons_list])
#     print(ngtv_neurons)
    
    # analyzed ngtv neuron zscore
    ngtv_neurons_zscore_list = ngtv_series_analyzed.values.tolist()
    ngtv_neurons_zscore = ','.join([str(i) for i in ngtv_neurons_zscore_list])
    print(ngtv_neurons_zscore)
    print(type(ngtv_neurons_zscore))
    
    # ngtv mean
    ngtv_mean = ngtv_series_analyzed.mean()
    
    # number of pstv be analyzed
    pstv_series_analyzed = pstv_series[round(len(pstv_series)*pstv_near):round(len(pstv_series)*(1-pstv_far))]
    pstv_analyzed_number = len(pstv_series_analyzed)
    
    # analyzed pstv neuron ID
    pstv_neurons_list = pstv_series_analyzed.index.to_list()
    pstv_neurons=','.join([str(i) for i in pstv_neurons_list])
#     print(pstv_neurons)
    
    # analyzed pstv neuron zscore
    pstv_neurons_zscore_list = pstv_series_analyzed.values.tolist()
    pstv_neurons_zscore = ','.join([str(i) for i in pstv_neurons_zscore_list])
#     print(pstv_neurons_zscore)
#     print(type(pstv_neurons_zscore))
    
    # pstv mean
    pstv_mean = pstv_series_analyzed.mean()
    
    out = pd.Series([total,ngtv_number,ngtv_near,ngtv_far,ngtv_analyzed_number,ngtv_neurons,ngtv_neurons_zscore,ngtv_mean,pstv_number,pstv_near,pstv_far,pstv_analyzed_number,pstv_neurons,pstv_neurons_zscore,pstv_mean],index=["total neuron","ngtv neuron","near (%) in the ngtv side", "far (%) in the ngtv side","ngtv neuron to be analyzed","analyzed ngtv neuron ID","analyzed ngtv zscore","negative mean","pstv neuron","near (%) in the pstv side","far (%) in the pstv side","pstv neuron to be analyzed","analyzed pstv neuron ID","analyzed pstv zscore","positive mean"])
#     out = pd.concat([Series,out])
    return out

In [ ]:
AUC_zscore_calculate_version1 = AUC_zscore_summary.iloc[:,:-3].apply(calculate_mean,ngtv_near=0.1,pstv_near=0.1,axis=1)
AUC_zscore_summary_version1 = pd.concat([AUC_zscore_summary,AUC_zscore_calculate_version1],axis=1)
AUC_zscore_summary_version1.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_AUC_zscore_summary_version1.csv"))

In [ ]:
AUC_zscore_calculate_version2 = AUC_zscore_summary.iloc[:,:-3].apply(calculate_mean,ngtv_near=0.2,pstv_near=0.2,axis=1)
AUC_zscore_summary_version2 =pd.concat([AUC_zscore_summary, AUC_zscore_calculate_version2],axis=1)
AUC_zscore_summary_version2.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_AUC_zscore_summary_version2.csv"))

## AUC of S per Event

In [ ]:
column_name = new_pd.columns.values  #get column label
print(column_name)
print(column_name[:-8])

In [ ]:
mark = np.zeros(column_name.size-8)
flag = np.zeros(column_name.size-8)

# for i in column_name[:-7]:
#     print(str(i)+": "+str(np.argwhere(new_pd.columns.values==i)[0][0]))


for i in column_name[:-8]:
    new_pd.insert(int(np.argwhere(new_pd.columns.values==i)[0][0]) + 1,str(i)+'_mark',0)   

for index,row in new_pd.iterrows():
    for i in range(column_name.size-8):
        if row[column_name[i]] == 0 and flag[i] != 0:
            flag[i]=0
        elif row[column_name[i]] == 0 and flag[i] == 0:
            continue
        elif row[column_name[i]] != 0 and flag[i] != 0:   
            new_pd.loc[index,str(column_name[i])+'_mark'] = mark[i]
        else:
            flag[i]=1
            mark[i]=mark[i]+1
            new_pd.loc[index,str(column_name[i])+'_mark'] = mark[i]



In [ ]:
new_pd

In [ ]:
# print(column_name)
print(new_pd.columns.values)

In [ ]:
for i in range(column_name.size-9,-1,-1):
    SUM = pd.DataFrame()
    SUM[str(column_name[i])+'_sum'] = new_pd[column_name[i]].groupby(new_pd[str(column_name[i])+'_mark']).sum()
    temp = pd.merge(new_pd[str(column_name[i])+'_mark'],SUM,how='left',left_on=str(column_name[i])+'_mark',right_index=True)
#     print(temp)
#     print(type(new_pd[column_name[i]].groupby(new_pd[str(column_name[i])+'_mark']).sum()))
    new_pd.insert(int(np.argwhere(new_pd.columns.values==str(column_name[i])+'_mark')[0][0])+1,str(column_name[i])+'_sum',0)
    new_pd.update(temp)

new_pd.insert(int(np.argwhere(new_pd.columns.values == 'Time Stamp (ms)')[0][0])+1,'Time Interval (ms)',new_pd['Time Stamp (ms)'].diff())
new_pd.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_AUC_of_S_per_event.csv"))

In [ ]:
# print(new_pd[2].groupby(new_pd['2_mark']).count())

## Frequency

In [ ]:
behavior_df=new_pd.loc[:,["min","RNFS","ALP","IALP","ALP_Timeout"]]
behavior_df2=behavior_df.groupby(behavior_df["min"]).sum()
behavior_df2

In [ ]:
df_min=new_pd['min']
frequency_df=new_pd.diff()
col=[]
for i in column_name[:-8]:
    col.append(str(i)+'_mark')
frequency_df=frequency_df[col]
frequency_df[frequency_df<0]=0
frequency_df2=frequency_df[frequency_df>0]
frequency_df2['min']=df_min
frequency_df2 = frequency_df2.groupby('min').count()
frequency_df2.columns=frequency_df2.columns.str.rstrip('_mark').astype('int64')
frequency_df3=pd.merge(frequency_df2,behavior_df2,left_index=True, right_on="min")
frequency_df3.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Number_of_S_events_per_min.csv"))
frequency_df3.columns.values

In [ ]:
frequency_df3

In [ ]:
Freq_zscore = calculate_zscore(session, frequency_df3)
Freq_zscore.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Freq_zscore.csv"))

In [ ]:
zscore_summary_index_list = list(filter(lambda x: type(x) != int,Freq_zscore.index.tolist()))
Freq_zscore_summary = Freq_zscore.loc[zscore_summary_index_list,:]

In [ ]:
Freq_zscore_calculate_version1 = Freq_zscore_summary.iloc[:,:-4].apply(calculate_mean,ngtv_near=0.1,pstv_near=0.1,axis=1)
Freq_zscore_summary_version1 = pd.concat([Freq_zscore_summary, Freq_zscore_calculate_version1],axis=1)
Freq_zscore_summary_version1.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Freq_zscore_summary_version1.csv"))

In [ ]:
Freq_zscore_calculate_version2 = Freq_zscore_summary.iloc[:,:-4].apply(calculate_mean,ngtv_near=0.2,pstv_near=0.2,axis=1)
Freq_zscore_summary_version2 = pd.concat([Freq_zscore_summary,Freq_zscore_calculate_version2],axis=1)
Freq_zscore_summary_version2.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Freq_zscore_summary_version2.csv"))

## Amp

In [ ]:
neuron_ids = res.columns.values.tolist()
neuron_ids = list(filter(lambda x:type(x) == int,neuron_ids))

In [ ]:
if session =="S1":
    amp_df=res.loc[0:14,neuron_ids]/frequency_df2.loc[0:14,:]
elif session == "S4":
    amp_df=res.loc[45:59,neuron_ids]/frequency_df2.loc[45:59,:]
amp_df=amp_df.fillna(0)
amp_df=amp_df.replace(np.inf,0)
# amp_df
if session =="S1":
    amp_df=pd.merge(amp_df,res.loc[0:14,["RNFS","ALP","IALP","ALP_Timeout"]],left_index=True,right_index=True)
elif session == "S4":
    amp_df=pd.merge(amp_df,res.loc[45:59,["RNFS","ALP","IALP","ALP_Timeout"]],left_index=True,right_index=True)
amp_df.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_average_Amp_per_event.csv"))

In [ ]:
Amp_zscore = calculate_zscore(session, amp_df)
Amp_zscore.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Amp_zscore.csv"))

In [ ]:
zscore_summary_index_list = list(filter(lambda x: type(x) != int,Amp_zscore.index.tolist()))
Amp_zscore_summary = Amp_zscore.loc[zscore_summary_index_list,:]

In [ ]:
Amp_zscore_calculate_version1 = Amp_zscore_summary.iloc[:,:-4].apply(calculate_mean,ngtv_near=0.1,pstv_near=0.1,axis=1)
Amp_zscore_summary_version1 = pd.concat([Amp_zscore_summary, Amp_zscore_calculate_version1],axis=1)
Amp_zscore_summary_version1.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Amp_zscore_summary_version1.csv"))

In [ ]:
Amp_zscore_calculate_version2 = Amp_zscore_summary.iloc[:,:-4].apply(calculate_mean,ngtv_near=0.2,pstv_near=0.2,axis=1)
Amp_zscore_summary_version2 = pd.concat([Amp_zscore_summary,Amp_zscore_calculate_version2],axis=1)
Amp_zscore_summary_version2.to_csv(os.path.join(output_path, mouseID+'_'+day+'_'+session+"_Amp_zscore_summary_version2.csv"))

# Frame check

In [ ]:
client.close()
cluster.close()